In [16]:
#!pip install transformers -q
#!pip install datasets -q
#!pip install langchain -q
#!pip install huggingface_hub -q
#!pip install accelerate -q#
#!pip install openai -q
#!pip install ftfy -q
#!pip install pyserini -q
!pip install faiss-cpu -q

In [4]:
import os
os.chdir('/content/drive/MyDrive/doutorado/P_IA368DD_2023S/aula8_9')

import numpy as np

np.random.seed(42)

path_data = "./data"

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
OPENAI_API_KEY = getpass()

··········
··········


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
#workdir = '/content/gdrive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks'


In [3]:
import pandas as pd
# import torch.nn.functional as F
import json
import time
#import ftfy

In [ ]:
from datasets import load_dataset
from langchain import (
    HuggingFacePipeline,
    HuggingFaceHub,
    PromptTemplate, 
    LLMChain
)
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
# from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from collections import defaultdict
from pyserini.search.lucene import LuceneSearcher
# from evaluate import load

In [17]:
from pyserini.search.lucene import LuceneSearcher

In [ ]:
trec_covid_queries = load_dataset("BeIR/trec-covid", 'queries')
trec_covid_corpus = load_dataset("BeIR/trec-covid", 'corpus')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
queries_ids = np.random.randint(len(trec_covid_corpus['corpus']), 
                                size = 1000)
test_queries_ids = np.random.randint(len(trec_covid_corpus['corpus']), 
                                size = 5)

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo',
                 model_kwargs={'temperature':1e-10})
#llama_model = 'decapoda-research/llama-7b-hf'
#flan_model = "google/flan-t5-xl"
#llm = HuggingFaceHub(repo_id=flan_model,
#                          model_kwargs={'temperature':1e-10,
#                                        "max_length": 64})

template = """Given this document, formulate one query for it.

{text}"""


prompt = PromptTemplate(template = template, 
                        input_variables = ["text"])

llm_chain = LLMChain(prompt = prompt, llm = llm)

In [ ]:
text = trec_covid_corpus['corpus']['text'][0]

print(llm_chain.run({'text': text}))

What were the epidemiology and clinical features of patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital in Jeddah, Saudi Arabia from January 1997 to December 1998?


In [ ]:
MAX_REQUEST_PER_MINUTE = 50

def generate_queries(llm_model, text):

  generated_query = llm_model.run({'text': text})
  return generated_query

def call_generate_queries(llm_model, samples, save_file_path):
  
  request_count = 0
  df = pd.DataFrame()
  generated_queries = []
  pos_doc_ids = []

  with open(f"{save_file_path}.jsonl", 'a', encoding='utf-8') as arquivo:

    for sample in tqdm(samples[335+187:]):
      #title = ftfy.fix_text(trec_covid_corpus['corpus']['title'][sample])
      text = ftfy.fix_text(trec_covid_corpus['corpus']['text'][sample])
      
      generated_query = generate_queries(llm_model, text)
      generated_queries.append(generated_query)
      pos_doc_ids.append(trec_covid_corpus['corpus']['_id'][sample])

      doc = {"positive_doc_id": trec_covid_corpus['corpus']['_id'][sample], "query": generated_query}
      arquivo.write(f"{json.dumps(doc)}\n")
      request_count += 1

      if request_count == MAX_REQUEST_PER_MINUTE:
        print(f"{request_count} requests. Sleep")
        time.sleep(5)
        request_count = 0

  df['query'] = generated_queries
  df['positive_doc_id'] = pos_doc_ids

  df.to_csv(f"{save_file_path}.csv")
  print(f'Saved to {save_file_path}')
  return df


if not os.path.exists(f"{path_data}"):
    os.mkdir(path_data)
    print("Pasta criada!")
else:
    print("Pasta já existe")

#test_generated_queries = call_generate_queries(llm_chain, test_queries_ids, f'{path_data}/test_queries.csv')
generated_queries = call_generate_queries(llm_chain, queries_ids, f'{path_data}/generated_queries')

In [8]:
generated_queries = pd.read_json(f'{path_data}/generated_queries.jsonl',lines=True)

In [9]:
generated_queries

,positive_doc_id,query
0,v8zyecxz,What compounds from Curcuma longa L. were foun...
1,tthrpxx6,What is the prevalence of RSV subtypes and dis...
2,tb0wl19x,What are the factors that contribute to the em...
3,b72m9x9j,What is the risk status of patients of a natio...
4,v8zyecxz,What compounds from Curcuma longa L. were foun...
...,...,...
540,g2eqaizz,"I'm sorry, but I cannot formulate a query with..."
541,4q9jrpop,What is the prevalence of human parainfluenza ...
542,hcknigdu,"I'm sorry, but I cannot formulate a query with..."
543,vdzvj0vx,"What are the clinical characteristics, prevent..."


In [11]:
# Inspired by Manoel Veríssimo dos Santos Neto
# https://github.com/verissimomanoel/P_IA368DD_2023S1/blob/main/Exercicio8/generate_dataset.py

def generate_ramdom_numbers(max = 5, k = 1000):
    random_list = []
    while len(random_list) < max:
        n = np.random.randint(0, k)

        # Prevent duplicated index
        if n not in random_list:
            random_list.append(n)

    return random_list

def search_with_bm25(query, max = 5, k = 1000):
    searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
    hits = searcher.search(query, k)
    random_list = generate_ramdom_numbers(max = max, k = k)
    random_ids = []

    for index in random_list:
        jsondoc = json.loads(hits[index].raw)
        random_ids.append(jsondoc["_id"])

    return random_ids

In [18]:
generated_queries['negative_doc_ids'] = generated_queries['query'].apply(search_with_bm25)

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:03, 57.3MB/s]                           


In [24]:
#generated_queries.drop('Unnamed: 0', axis = 1, inplace = True)
generated_queries

,positive_doc_id,query,negative_doc_ids
0,v8zyecxz,What compounds from Curcuma longa L. were foun...,"[i0mlsxkn, s71naf96, h23iqb7s, 6zk0ioep, wevcm..."
1,tthrpxx6,What is the prevalence of RSV subtypes and dis...,"[csjowuwt, oxsocl1c, mrxgcg6z, 3i90blvy, 1c630..."
2,tb0wl19x,What are the factors that contribute to the em...,"[ymwqp6e2, tzv2ntev, hvvyeeuj, 49it5pha, t3iou..."
3,b72m9x9j,What is the risk status of patients of a natio...,"[t1xf06tl, v5tlpp4b, ou7hpqjd, eaac0cem, zbjgr..."
4,v8zyecxz,What compounds from Curcuma longa L. were foun...,"[p77lyofz, mj2t1iwb, 6rs95axb, kcyao9i9, v48tl..."
...,...,...,...
540,g2eqaizz,"I'm sorry, but I cannot formulate a query with...","[uyk76b3t, hlhhvmtu, x3o3a45b, tqn9j322, iwk60..."
541,4q9jrpop,What is the prevalence of human parainfluenza ...,"[e32nybra, birsn4kx, 08kqtw8s, zipc9zxw, qrdhh..."
542,hcknigdu,"I'm sorry, but I cannot formulate a query with...","[pb2ydrt4, xyqzp4h1, wboueeti, em7pg8mj, 6frnb..."
543,vdzvj0vx,"What are the clinical characteristics, prevent...","[w8gjy2nk, 2gbxtt7l, 41ycjpt9, aiel1i0u, 5jsch..."


In [23]:
generated_queries.to_json(f'{path_data}/leodeciobraz_545_queries.jsonl', lines = True, orient = 'records')